# Black-Scholes Implied Volatility Function

## Theory

The Black-Scholes model is a fundamental tool in options pricing. It provides a theoretical estimate of the price of European-style options. The model assumes that:

1. The stock price follows a geometric Brownian motion with constant drift and volatility.
2. There are no transaction costs or taxes.
3. The risk-free interest rate is constant.
4. The stock does not pay a dividend.

The Black-Scholes formula for a European call option is:

C = S₀N(d₁) - Ke^(-rτ)N(d₂)

Where:
- C is the call option price
- S₀ is the current stock price
- K is the strike price
- r is the risk-free interest rate
- τ is the time to maturity
- N(x) is the cumulative distribution function of the standard normal distribution
- d₁ = (ln(S₀/K) + (r + σ²/2)τ) / (σ√τ)
- d₂ = d₁ - σ√τ

The implied volatility is the volatility that, when used in the Black-Scholes formula, gives a theoretical price equal to the market price of the option. It's called "implied" because it's derived from the market price, rather than being directly observable.

## Code Explanation

In [63]:
import  numpy as np 
import scipy.stats as st


# intialization of parameters

In [118]:
V_market = 2    # market call option price
K        = 120  # strike
tau      = 1    # time-to-maturity
r        = 0.05 # interest rate
S_0      = 100  # today's stock price
sigmaInit    = 0.25  # Initial implied volatility
CP       ="c" #C is call and P is put

In [119]:
def ImpliedVolatility(CP,S_0,K,sigma,tau,r):
    error    = 1e10; # initial error
    #Handy lambda expressions
    optPrice = lambda sigma: BS_Call_Option_Price(CP,S_0,K,sigma,tau,r)
    vega= lambda sigma: dV_dsigma(S_0,K,sigma,tau,r)

    n = 1.0 
    while error>10e-10:
        g         = optPrice(sigma) - V_market
        g_prim    = vega(sigma)
        sigma_new = sigma - g / g_prim
        #error=abs(sigma_new-sigma)
        error=abs(g)
        sigma=sigma_new;   
        print('iteration {0} with error = {1}'.format(n,error))
        n= n+1
    return sigma

This function uses the Newton-Raphson method to find the implied volatility. It iteratively refines the volatility estimate until the calculated option price matches the market price within a small error threshold.

Newton-Raphson Step:
Compute the error (g) between the model option price and the market price.
Adjust 
𝜎
σ by subtracting the ratio of the error to Vega:
𝜎
new
=
σ 
new
​
 =σ− 
g/vega

​



The helper functions are defined as follows:

In [120]:
def dV_dsigma(S_0,K,sigma,tau,r):
    #parameters and value of Vega
    d2   = (np.log(S_0 / float(K)) + (r - 0.5 * np.power(sigma,2.0)) * tau) / float(sigma * np.sqrt(tau))
    value = K * np.exp(-r * tau) * st.norm.pdf(d2) * np.sqrt(tau)
    return value
def BS_Call_Option_Price(CP,S_0,K,sigma,tau,r):
    #Black-Scholes Call option price
    d1    = (np.log(S_0 / float(K)) + (r + 0.5 * np.power(sigma,2.0)) * tau) / float(sigma * np.sqrt(tau))
    d2    = d1 - sigma * np.sqrt(tau)
    if str(CP).lower()=="c" or str(CP).lower()=="1":
        value = st.norm.cdf(d1) * S_0 - st.norm.cdf(d2) * K * np.exp(-r * tau)
    elif str(CP).lower()=="p" or str(CP).lower()=="-1":
        value = st.norm.cdf(-d2) * K * np.exp(-r * tau) - st.norm.cdf(-d1)*S_0
    return value

`dV_dsigma` calculates the option's vega (sensitivity to volatility), which is used in the Newton-Raphson method.

`BS_Call_Option_Price` implements the Black-Scholes formula for both call and put options.

Finally, we use these functions to calculate the implied volatility:

In [121]:
sigma_imp = ImpliedVolatility(CP,S_0,K,sigmaInit,tau,r)
message = '''Implied volatility for CallPrice= {}, strike K={}, 
      maturity T= {}, interest rate r= {} and initial stock S_0={} 
      equals to sigma_imp = {:.7f}'''.format(V_market,K,tau,r,S_0,sigma_imp)
            
print(message)

iteration 1.0 with error = 3.025413481792615
iteration 2.0 with error = 0.19134998568795325
iteration 3.0 with error = 0.0022254541477302325
iteration 4.0 with error = 3.353154056640051e-07
iteration 5.0 with error = 1.0658141036401503e-14
Implied volatility for CallPrice= 2, strike K=120, 
      maturity T= 1, interest rate r= 0.05 and initial stock S_0=100 
      equals to sigma_imp = 0.1614827


In [122]:
val = BS_Call_Option_Price(CP,S_0,K,sigma_imp,tau,r)
print('Option Price for implied volatility of {0} is equal to {1}'.format(sigma_imp, val))

Option Price for implied volatility of 0.1614827288413938 is equal to 2.0


This code calculates the implied volatility and then verifies the result by using the calculated implied volatility to price the option, which should match the original market price.

## Conclusion

This implementation demonstrates how to calculate implied volatility using the Black-Scholes model and the Newton-Raphson method. It's a crucial tool in options trading and risk management, as implied volatility provides insights into market expectations of future stock price volatility.

In [18]:
((r-(sigma*sigma)/2.0)*tau)

0.018750000000000003

In [19]:
((r - 0.5 * np.power(sigma,2.0)) * tau)

0.018750000000000003

In [25]:
d2=(np.log(S_0/float(K))+((r-(sigma*sigma)/2)*tau)/float(sigma*np.sqrt(tau)))

45.27677696686181

45.27677696686181

In [94]:
#%%
"""
Created on Wed Oct 24 20:37:32 2018
Black- Scholes implied volatility function

@author: Lech A. Grzelak
"""
import numpy as np
import scipy.stats as st
# Initial parameters and market quotes
V_market = 2    # market call option price
K        = 120  # strike
tau      = 1    # time-to-maturity
r        = 0.05 # interest rate
S_0      = 100  # today's stock price
sigmaInit    = 0.40  # Initial implied volatility
CP       ="c" #C is call and P is put

def ImpliedVolatility(CP,S_0,K,sigma,tau,r):
    error    = 1e10; # initial error
    #Handy lambda expressions
    optPrice = lambda sigma: BS_Call_Option_Price(CP,S_0,K,sigma,tau,r)
    vega= lambda sigma: dV_dsigma(S_0,K,sigma,tau,r)
    
    # While the difference between the model and the arket price is large
    # follow the iteration
    n = 1.0 
    while error>10e-10:
        g         = optPrice(sigma) - V_market
        g_prim    = vega(sigma)
        sigma_new = sigma - g / g_prim
    
        #error=abs(sigma_new-sigma)
        error=abs(g)
        sigma=sigma_new;
        
        print('iteration {0} with error = {1}'.format(n,error))
        
        n= n+1
    return sigma

# Vega, dV/dsigma
def dV_dsigma(S_0,K,sigma,tau,r):
    #parameters and value of Vega
    d2   = (np.log(S_0 / float(K)) + (r - 0.5 * np.power(sigma,2.0)) * tau) / float(sigma * np.sqrt(tau))
    value = K * np.exp(-r * tau) * st.norm.pdf(d2) * np.sqrt(tau)
    return value

def BS_Call_Option_Price(CP,S_0,K,sigma,tau,r):
    #Black-Scholes Call option price
    d1    = (np.log(S_0 / float(K)) + (r + 0.5 * np.power(sigma,2.0)) * tau) / float(sigma * np.sqrt(tau))
    d2    = d1 - sigma * np.sqrt(tau)
    if str(CP).lower()=="c" or str(CP).lower()=="1":
        value = st.norm.cdf(d1) * S_0 - st.norm.cdf(d2) * K * np.exp(-r * tau)
    elif str(CP).lower()=="p" or str(CP).lower()=="-1":
        value = st.norm.cdf(-d2) * K * np.exp(-r * tau) - st.norm.cdf(-d1)*S_0
    return value

sigma_imp = ImpliedVolatility(CP,S_0,K,sigmaInit,tau,r)
message = '''Implied volatility for CallPrice= {}, strike K={}, 
      maturity T= {}, interest rate r= {} and initial stock S_0={} 
      equals to sigma_imp = {:.7f}'''.format(V_market,K,tau,r,S_0,sigma_imp)
            
print(message)

# Check! 
val = BS_Call_Option_Price(CP,S_0,K,sigma_imp,tau,r)
print('Option Price for implied volatility of {0} is equal to {1}'.format(sigma_imp, val))

iteration 1.0 with error = 8.805973916005634
iteration 2.0 with error = 0.4971232418421323
iteration 3.0 with error = 0.012818322580802288
iteration 4.0 with error = 1.1054906480012505e-05
iteration 5.0 with error = 8.277822871605167e-12
Implied volatility for CallPrice= 2, strike K=120, 
      maturity T= 1, interest rate r= 0.05 and initial stock S_0=100 
      equals to sigma_imp = 0.1614827
Option Price for implied volatility of 0.16148272884139397 is equal to 1.9999999999999893
